In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import os
import re
import urllib3
import urllib.parse
from urllib.request import urlopen
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import time
from requests.auth import HTTPBasicAuth
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_status(logs):
    for log in logs:
        if log['message']:
            d = json.loads(log['message'])
            try:
                content_type = 'text/html' in d['message']['params']['response']['headers']['content-type']
                response_received = d['message']['method'] == 'Network.responseReceived'
                if content_type and response_received:
                    return d['message']['params']['response']['status']
            except:
                pass

def preprocess(line):
    return ' '.join(line.replace('\r',' ').replace('\xa0',' ').replace('\xad', ' ').strip().split())

def download_pdf(link,name):
    filename = Path(name)
    response = requests.get(url)
    filename.write_bytes(response.content)

def get_text(soup):
    text = []
    for script in soup(["script", "style","footer","input",'link','meta','button']):
        script.extract()

    paragraph = '\n'.join(list(dict.fromkeys([text for text in soup.stripped_strings])))
    return paragraph


def get_all_links(url,link_hierarchy,soup):
    child_links = []
    for script in soup(["script", "style"]):
        script.extract()
    
    sublinks = [a.get('href') for a in soup.find_all('a') if a.get('href') and a.get('href').startswith('/')]
    
    sublinks = [link[1:] for link in sublinks]
    sublinks = [*set(sublinks)]
    sublinks = list(filter(None, sublinks))
    
    for link in sublinks:
        if ".pdf" not in link or ".PDF" not in link:
            if 'www' in link:
                new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
            else:
                new_url = os.path.join(url[:url.rfind('//')+2], urlparse(url).netloc, link)
            child_links.append(new_url)
    
    link_hierarchy[url] = child_links
    return

def get_responce(links):
    responses = []
    selenium = []
    status = 0
    page = ''
    header = {
  'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
  'accept-encoding':'gzip, deflate, br',
  'accept-language':'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
  'cache-control':'no-cache',
  'dnt': '1',
  'pragma': 'no-cache',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'none',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
    }
    for link in links:
        try:
            page = requests.get(link,headers=header,auth=HTTPBasicAuth('user', 'pass'),verify=False)
            status = page.status_code
        except:
            print('Cannot reach website')
        if status == 200:
            page.encoding = 'utf-8'
            responses.append((link, page.text))
            print(link, status, sep=' : ')
        else:
            page.encoding = 'utf-8'
            selenium.append(link)
            print(link, status, sep=' : ')
        status = 0
    return responses

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
driver = webdriver.Chrome(options = chrome_options)

In [4]:
DataSet = pd.read_csv('ESG.csv')

In [5]:
DataSet = DataSet.drop(['Unnamed: 15'],axis=1)
DataSet

,Бренд и Юрлицо (!),Веб-сайт,ИНН,МOEX,Регион,"Устойчивое развитие, Социальная отчетность, ESG",Экология,"Забота о потребителях, защита прав потребителей","Забота о работниках, социальные программы","Поставщикам, Закупки","Корпоративное управление, совет директоров, высший менеджмент","Миссия, ценности",Этический кодекс,Антикоррупция,Раскрытие информации,Количесво ссылок
0,Адмиралтейские верфи,http://admship.ru/,7839395419,NaN,СЗФО,нет,нет,нет,http://admship.ru/job/,http://admship.ru/purchase/,нет,http://admship.ru/about/corruption/,нет,http://admship.ru/about/corruption/,http://admship.ru/about/disclosure/,5.0
1,Алмаз-Антей,http://www.almaz-antey.ru/,7731084175,NaN,СЗФО,нет,нет,нет,http://www.almaz-antey.ru/kadrovaya-i-sotsialn...,http://www.almaz-antey.ru/zakupochnaya-deyatel...,http://www.almaz-antey.ru/istoriya/,http://www.almaz-antey.ru/strategiya/,http://www.almaz-antey.ru/upload/corruption/%D...,http://www.almaz-antey.ru/anti-corruption/,нет,7.0
2,Армалит,https://armalit.ru/,7805148130,NaN,СЗФО,нет,нет,нет,https://armalit.ru/about/training-center/,нет,https://armalit.ru/about/management/,нет,нет,нет,https://armalit.ru/information-disclosure/,3.0
3,Аттика,https://attikarus.ru/,7806521932,NaN,СЗФО,нет,нет,нет,нет,https://attikarus.ru/about/suppliers/,https://attikarus.ru/about/team/,https://attikarus.ru/about/history/,нет,https://attikarus.ru/anti-corruption-policy/,нет,5.0
4,Балтийский завод,https://www.bz.ru/,7830001910,NaN,СЗФО,нет,нет,нет,https://www.bz.ru/career/social-policy/,https://www.bz.ru/partners/purchases/,https://www.bz.ru/about/ceo/,нет,нет,https://www.bz.ru/protivodeystvie-korruptsii/,https://www.bz.ru/partners/disclosure-of-infor...,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,«Эфирное»,https://www.efko.ru,3122000300,NaN,RAEX,нет,Нет,Нет,https://www.efko.ru/career/,Нет,нет,https://www.efko.ru/about/#mission,нет,нет,Нет,3.0
305,«Торговая компания «Мираторг»,http://www.miratorg.ru/,5009072150,NaN,RAEX,https://miratorg.ru/about/sustainable_developm...,Нет,Нет,https://miratorg.ru/career/vacancies/,Нет,https://miratorg.ru/about/directors/,нет,нет,нет,https://e-disclosure.ru/portal/company.aspx?id...,4.0
306,РОСВОДОКАНАЛ,http://www.rosvodokanal.ru/,5027116065,NaN,RAEX,https://www.rosvodokanal.ru/ustoychivoe-razvit...,https://www.rosvodokanal.ru/ustoychivoe-razvit...,https://www.rosvodokanal.ru/ustoychivoe-razvit...,https://www.rosvodokanal.ru/ustoychivoe-razvit...,https://www.rosvodokanal.ru/about/zakupki/,https://www.rosvodokanal.ru/about/rukovodstvo/,https://www.rosvodokanal.ru/about/missiya/,нет,нет,https://www.rosvodokanal.ru/ustoychivoe-razvit...,8.0
307,«Мосводоканал»,http://www.mosvodokanal.ru/,7701984274,NaN,RAEX,https://www.mosvodokanal.ru/about/evolution/ec...,https://www.mosvodokanal.ru/about/evolution/ec...,https://www.mosvodokanal.ru/forpeople/waterqua...,нет,https://www.mosvodokanal.ru/sales/,https://www.mosvodokanal.ru/about/leadership/,нет,нет,https://www.mosvodokanal.ru/about/anti-corrupt...,https://www.mosvodokanal.ru/about/disclosure/,7.0


In [ ]:
responses = get_responce(list(DataSet['Веб-сайт'].values))

http://admship.ru/ : 200
http://www.almaz-antey.ru/ : 200
https://armalit.ru/ : 200
https://attikarus.ru/  : 404
https://www.bz.ru/ : 200
https://www.bronkagroup.ru/ : 200
https://werker.ru/ : 200
https://www.gidropribor.ru/ : 200
https://www.granit-electron.ru/ : 200
Cannot reach website
https://www.izmeron.ru/ : 0
https://www.ipm.ru/ : 200
https://kzgroup.ru/ : 200
https://aokmp.ru/ : 200
https://meteor.ru/ : 200
https://www.sitronics.com/ : 200
https://t-pack.ru/ : 200
Cannot reach website
https://www.star.ru/ : 0
http://www.elektropribor.spb.ru/ : 200
https://www.polyplastic.ru : 200
https://www.packaging-systems.ru : 200
https://ecoprofchem.ru/ : 200
https://www.phosagro.ru : 200
https://www.uralkali.com/ru/ : 200
https://www.uralchem.ru : 200
https://www.acron.ru : 200
https://www.eurochem.ru : 200
https://www.toaz.ru : 200
https://avgust.com : 200
http://n-azot.ru : 200
https://kao-azot.com : 200
https://krata.ru : 200
https://www.kaustik.ru/ru/ : 200
https://metafrax.ru : 200
h

In [11]:
len(responses)

243

In [36]:
def parse(url):
    response = requests.get(url)
    if response.status_code == 200:
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        return get_text(soup)
    else:
        return ''

In [257]:
data = pd.DataFrame(columns=['Название','Ссылка','Устойчивое развитие, Социальная отчетность, ESG','E','S','G','оценка'])

In [258]:
data_error = pd.DataFrame(columns=['Название','Ссылка', 'Устойчивое развитие, Социальная отчетность, ESG','E','S','G'])

In [259]:
data_pdf = pd.DataFrame(columns=['Название','Ссылка', 'Устойчивое развитие, Социальная отчетность, ESG','E','S','G'])

In [260]:
#1
text_s1_admship = parse('http://admship.ru/job/')
text_s2_admship = parse('http://admship.ru/purchase/')
text_s_admship = text_s1_admship + text_s2_admship

text_g_admship = parse('http://admship.ru/about/corruption/')
data.loc[len(data.index)] = ['Адмиралтейские верфи', 'http://admship.ru/', 'NaN', 'NaN', text_s_admship, text_g_admship,'AAA']

In [261]:
#2
text_s1_almaz = parse('http://www.almaz-antey.ru/kadrovaya-i-sotsialnaya-politika/')
text_s2_almaz = parse('http://www.almaz-antey.ru/zakupochnaya-deyatelnost/')
text_s_almaz = text_s1_almaz + text_s2_almaz

text_g1_almaz = parse('http://www.almaz-antey.ru/istoriya/')
text_g2_almaz = parse('http://www.almaz-antey.ru/strategiya/')
text_g3_almaz = parse('http://www.almaz-antey.ru/anti-corruption/')
text_g_almaz = text_g1_almaz + text_g2_almaz + text_g3_almaz

data.loc[len(data.index)] = ['Алмаз-Антей', 'http://www.almaz-antey.ru/', 'NaN', 'NaN', text_s_almaz, text_g_almaz,'AAA']
data_pdf.loc[len(data_pdf.index)] = ['Алмаз-Антей', 'http://www.almaz-antey.ru/', 'NaN', 'NaN', 'NaN', 'http://www.almaz-antey.ru/upload/corruption/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D0%B8_%D0%B5%D0%B4%D0%B8%D0%BD%D1%8B%D0%B5_%D0%BF%D1%80%D0%B8%D0%BD%D1%86%D0%B8%D0%BF%D1%8B_%D0%BA%D0%BE%D1%80%D0%BF%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BF%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D1%8F.pdf']

In [262]:
#3
text_s_armalit = parse('https://armalit.ru/about/training-center/')

text_g_armalit = parse('https://armalit.ru/about/management/')

data.loc[len(data.index)] = ['Армалит', 'https://armalit.ru/', 'NaN', 'NaN', text_s_armalit, text_g_armalit,'AAA']

In [263]:
#4
text_s_attikarus = parse('https://attikarus.ru/about/suppliers/')

text_g1_attikarus = parse('https://attikarus.ru/about/team/')
text_g2_attikarus = parse('https://attikarus.ru/about/history/')
text_g3_attikarus = parse('https://attikarus.ru/anti-corruption-policy/')
text_g_attikarus = text_g1_attikarus + text_g2_attikarus + text_g3_attikarus

data.loc[len(data.index)] = ['Аттика', 'https://attikarus.ru/', 'NaN', 'NaN', text_s_attikarus, text_g_attikarus,'AAA']

In [264]:
#5
text_s1_bz = parse('https://www.bz.ru/career/social-policy/')
text_s2_bz = parse('https://www.bz.ru/partners/purchases/')
text_s_bz = text_s1_bz + text_s2_bz

text_g1_bz = parse('https://www.bz.ru/about/ceo/')
text_g2_bz = parse('https://www.bz.ru/protivodeystvie-korruptsii/')
text_g_bz = text_g1_bz + text_g2_bz

data.loc[len(data.index)] = ['Балтийский завод', 'https://www.bz.ru/', 'NaN', 'NaN', text_s_bz, text_g_bz,'AAA']

In [265]:
#6
text_e_bronkagroup = parse('https://www.bronkagroup.ru/about/sotsialnaya-otvetsvennost/')

text_s_bronkagroup = parse('https://www.bronkagroup.ru/career/')

text_g1_bronkagroup = parse('https://www.bronkagroup.ru/about/bod/')
text_g2_bronkagroup = parse('https://www.bronkagroup.ru/about/about-us/')
text_g_bronkagroup = text_g1_bronkagroup + text_g2_bronkagroup

data.loc[len(data.index)] = ['Бронка Групп', 'https://www.bronkagroup.ru/', 'NaN', text_e_bronkagroup, text_s_bronkagroup, text_g_bronkagroup,'AAA']

In [266]:
#7
text_s_werker = parse('https://werker.ru/about/vacancy/')

text_g_werker = parse('https://werker.ru/about/')

data.loc[len(data.index)] = ['Всеволожский крановый завод', 'https://werker.ru/', 'NaN', 'NaN', text_s_werker, text_g_werker, 'AAA']

In [267]:
#8
text_s1_gidropribor = parse('https://www.gidropribor.ru/personal/korporativnaya-kultura/')
text_s2_gidropribor = parse('https://www.gidropribor.ru/kontragents/zakupki/gidropribor-zakupki/')
text_s_gidropribor = text_s1_gidropribor + text_s2_gidropribor

text_g_gidropribor = parse('https://www.gidropribor.ru/kontragents/antikorrupcionnaya-politika/')

data.loc[len(data.index)] = ['Гидроприбор', 'https://www.gidropribor.ru/', 'NaN', 'NaN', text_s_gidropribor, text_g_gidropribor, 'AAA']

In [268]:
#9
text_s1_granit = parse('https://www.granit-electron.ru/staff/corporate-employment-benefits/')
text_s2_granit = parse('https://www.granit-electron.ru/concern/general-questions/purchase/')
text_s_granit = text_s1_granit + text_s2_granit

text_g_granit = parse('https://www.granit-electron.ru/concern/general-questions/documents/')

data.loc[len(data.index)] = ['Гранит-Электрон', 'https://www.granit-electron.ru/', 'NaN', 'NaN', text_s_granit, text_g_granit, 'AAA']

In [269]:
#10
data_error.loc[len(data_error.index)] = ['Завод «Измерон»', 'https://www.izmeron.ru/', 'NaN', 'NaN', 'NaN', 'NaN']

In [270]:
#11
data.loc[len(data.index)] = ['Императорский Фарфоровый завод', 'https://www.ipm.ru/', 'NaN', 'NaN', 'NaN', 'NaN', 'AAA']

In [271]:
#12
text_e_kz = parse('https://kzgroup.ru/companiya/')
text_s_kz = parse('https://kzgroup.ru/korporativnyy-universitet/')
text_g_kz = parse('https://kzgroup.ru/')

data.loc[len(data.index)] = ['Кировский завод', 'https://kzgroup.ru/', 'NaN', text_e_kz, text_s_kz, text_g_kz,'AAA']

In [272]:
#13
text_g1_aokmp = parse('https://aokmp.ru/')
text_g2_aokmp = parse('https://aokmp.ru/anticorruption/')
text_g_aokmp = text_g1_aokmp + text_g2_aokmp

data.loc[len(data.index)] = ['Корпорация морского приборостроения', 'https://aokmp.ru/', 'NaN', 'NaN', 'NaN', text_g_aokmp,'AAA']

In [273]:
#14
text_s_meteor = parse('https://meteor.ru/company/career/')
text_g_meteor = parse('https://meteor.ru/company/')

data.loc[len(data.index)] = ['Метеор лифт (бывший Отис)', 'https://meteor.ru/', 'NaN', 'NaN', text_s_meteor, text_g_meteor,'AAA']

In [274]:
#15
text_s1_sitronics = parse('https://www.sitronics.com/career')
text_s2_sitronics = parse('https://www.sitronics.com/procurement')
text_s_sitronics = text_s1_sitronics + text_s2_sitronics

text_g_sitronics = parse('https://www.sitronics.com/company')

data.loc[len(data.index)] = ['Ситроникс', 'https://www.sitronics.com/', 'NaN', 'NaN', text_s_sitronics, text_g_sitronics,'AAA']
data_pdf.loc[len(data_pdf.index)] = ['Ситроникс', 'https://www.sitronics.com/', 'NaN', 'NaN', 'NaN', 'https://www.sitronics.com/wp-content/uploads/2022/02/kodeks-delovogo-povedeniya-i-etiki.pdf']
data_pdf.loc[len(data_pdf.index)] = ['Ситроникс', 'https://www.sitronics.com/', 'NaN', 'NaN', 'NaN', 'https://www.sitronics.com/wp-content/uploads/2022/02/antikorrupczionnaya-politika.pdf']

In [275]:
#16
text_e_tpack = parse('https://t-pack.ru/social/')
text_s_tpack = parse('https://t-pack.ru/team/')
text_g_tpack = parse('https://t-pack.ru/about/')

data.loc[len(data.index)] = ['Транспак', 'https://t-pack.ru/', 'NaN', text_e_tpack, text_s_tpack, text_g_tpack,'AAA']

In [276]:
#17
data_error.loc[len(data_error.index)] = ['Центр судоремонта "Звёздочка"', 'https://www.star.ru/', 'NaN', 'NaN', 'NaN', 'NaN']

In [277]:
#18
text_s1_elektropribor = parse('http://www.elektropribor.spb.ru/kadrovaya-politika/')
text_s2_elektropribor = parse('http://www.elektropribor.spb.ru/o-predpriyatii/zakupki/')
text_s_elektropribor = text_s1_elektropribor + text_s2_elektropribor

text_g1_elektropribor = parse('http://www.elektropribor.spb.ru/o-predpriyatii/generalnyy-direktor/')
text_g2_elektropribor = parse('http://www.elektropribor.spb.ru/o-predpriyatii/ofitsialnaya-informatsiya/antikorruptsionnaya-politika/')
text_g_elektropribor = text_g1_elektropribor + text_g2_elektropribor

data.loc[len(data.index)] = ['Электроприбор', 'http://www.elektropribor.spb.ru/', 'NaN', 'NaN', text_s_elektropribor, text_g_elektropribor,'AAA']
data_pdf.loc[len(data_pdf.index)] = ['Электроприбор', 'http://www.elektropribor.spb.ru/', 'NaN', 'NaN', 'NaN', 'http://www.elektropribor.spb.ru/upload/medialibrary/e95/doc2.pdf']

In [278]:
#19
data_error.loc[len(data_error.index)] = ['«Группа ПОЛИПЛАСТИК»', 'https://www.polyplastic.ru', 'NaN', 'NaN', 'NaN', 'NaN']
data_pdf.loc[len(data_pdf.index)] = ['«Группа ПОЛИПЛАСТИК»', 'https://www.polyplastic.ru', 'NaN', 'NaN', 'NaN', 'https://www.polyplastic.ru/upload/company/anticorrupt/antikorrupcionnaya-politika-ooo-grpp.pdf']

In [279]:
#20
text_esg_packaging = parse('https://www.packaging-systems.ru/sustainable/')
text_e_packaging = parse('https://www.packaging-systems.ru/sustainable/')
text_g_packaging = parse('https://www.packaging-systems.ru/#missions')

data.loc[len(data.index)] = ['«Упаковочные системы»', 'https://www.packaging-systems.ru', text_esg_packaging, text_e_packaging, 'NaN', text_g_packaging, 'AAA']

In [280]:
#21
text_g_ecoprofchem = parse('https://ecoprofchem.ru/about/')

data.loc[len(data.index)] = ['ЭкопрофХим', 'https://ecoprofchem.ru/', 'NaN', 'NaN', 'NaN', text_g_ecoprofchem, 'AAA']

In [281]:
#22
text_esg_phosagro = parse('https://www.phosagro.ru/sustainability/')
text_e_phosagro = parse('https://www.phosagro.ru/sustainability/ecology/')

text_s1_phosagro = parse('https://www.phosagro.ru/production/attention/')
text_s2_phosagro = parse('https://www.phosagro.ru/sustainability/social-response/')
text_s3_phosagro = parse('https://www.phosagro.ru/procurement/')
text_s_phosagro = text_s1_phosagro + text_s2_phosagro + text_s3_phosagro

text_g1_phosagro = parse('https://www.phosagro.ru/about/management/')
text_g2_phosagro = parse('https://www.phosagro.ru/sustainability/')
text_g_phosagro = text_g1_phosagro + text_g2_phosagro

data.loc[len(data.index)] = ['«ФосАгро»', 'https://www.phosagro.ru', text_esg_phosagro, text_e_phosagro, text_s_phosagro, text_g_phosagro, 'AAA']
data_pdf.loc[len(data_pdf.index)] = ['«ФосАгро»', 'https://www.phosagro.ru', 'NaN', 'NaN', 'NaN', 'https://ar2022.phosagro.ru/pdf/ar/ru/corporate-governance_ethical-practices.pdf']

In [282]:
#23
data_error.loc[len(data_error.index)] = ['«Уралкалий»', 'https://www.uralkali.com/ru/', 'NaN', 'NaN', 'NaN', 'NaN']

In [283]:
#24
text_esg_uralchem = parse('https://www.uralchem.ru/esg/')
text_e_uralchem = parse('https://www.uralchem.ru/esg/ekologicheskiy-aspekt-e/')

text_s1_uralchem = parse('https://www.uralchem.ru/products/vnimaniyu-pokupateley/')
text_s2_uralchem = parse('https://www.uralchem.ru/esg/sotsialnyy-aspekt-s/')
text_s3_uralchem = parse('https://www.uralchem.ru/purchase/kak-stat-postavshchikom/')
text_s_uralchem = text_s1_uralchem + text_s2_uralchem + text_s3_uralchem

text_g1_uralchem = parse('https://www.uralchem.ru/corporate_management/')
text_g2_uralchem = parse('https://www.uralchem.ru/about/mission/')
text_g3_uralchem = parse('https://www.uralchem.ru/corporate_management/antikorruptsionnye-programmy/polozhenie-ob-antikorruptsionnoy-politike-ao-okhk-uralkhim/')
text_g_uralchem = text_g1_uralchem + text_g2_uralchem + text_g3_uralchem

data.loc[len(data.index)] = ['«Уралхим», ОХК', 'https://www.uralchem.ru', text_esg_uralchem, text_e_uralchem, text_s_uralchem, text_g_uralchem, 'AAA']

In [284]:
#25
data_error.loc[len(data_error.index)] = ['«Акрон», группа', 'https://www.acron.ru', 'NaN', 'NaN', 'NaN', 'NaN']

In [285]:
#26
text_esg_eurochem = parse('https://www.eurochem.ru/sustainability/')
text_e_eurochem = parse('https://www.eurochem.ru/sustainability/communities/')

text_s_eurochem = parse('https://www.eurochem.ru/sustainability/communities/')

text_g_eurochem = parse('https://www.eurochem.ru/corporate-governance/')

data.loc[len(data.index)] = ['МХК «ЕвроХим»', 'https://www.eurochem.ru', text_esg_eurochem, text_e_eurochem, text_s_eurochem, text_g_eurochem, 'AAA']

In [286]:
#27
text_e_toaz = parse('https://www.toaz.ru/social-responsibility/environment')

text_s1_toaz = parse('https://www.toaz.ru/product/liniya-doveriya')
text_s2_toaz = parse('https://www.toaz.ru/social-responsibility/charity-and-support-of-the-region/')
text_s3_toaz = parse('https://www.toaz.ru/zakupki/informacziya-dlya-postavshhikov')
text_s_toaz = text_s1_toaz + text_s2_toaz + text_s3_toaz

text_g_toaz = parse('https://www.toaz.ru/korporativnoe-upravlenie')

data.loc[len(data.index)] = ['«Тольяттиазот»', 'https://www.toaz.ru', 'NaN', text_e_toaz, text_s_toaz, text_g_toaz, 'AAA']
data_pdf.loc[len(data_pdf.index)] = ['«Тольяттиазот»', 'https://www.toaz.ru', 'NaN', 'NaN', 'NaN', 'https://ar2022.phosagro.ru/pdf/ar/ru/corporate-governance_ethical-practices.pdf']

In [287]:
#28

#выводит пустое значение
#text_esg_avgust = parse('https://avgust.com/services/newspaper/nasha_zadacha_ustoychivoe_razvitie')

text_s_avgust = parse('https://avgust.com/company/social-responsibility/')
text_g_avgust = parse('https://avgust.com/company/strategy/business/')

data.loc[len(data.index)] = ['«Фирма «Август»', 'https://avgust.com', 'NaN', 'NaN', text_s_avgust, text_g_avgust, 'AAA']
data_pdf.loc[len(data_pdf.index)] = ['«Фирма «Август»', 'https://avgust.com', 'sustainability-report-toaz.pdf', 'NaN', 'NaN', 'NaN']

In [288]:
#29
text_e_azot = parse('https://n-azot.ru/responsibility.php?pos=20')

text_s1_azot = parse('https://n-azot.ru/responsibility.php?pos=28')
text_s2_azot = parse('https://n-azot.ru/tender.php')
text_s_azot = text_s1_azot + text_s2_azot

text_g_azot = parse('https://n-azot.ru/info.php?lang=RU&group=386')

data.loc[len(data.index)] = ['«Щёкиноазот»', 'http://n-azot.ru', 'NaN', text_e_azot, text_s_azot, text_g_azot, 'AAA']

In [289]:
#30
data_error.loc[len(data_error.index)] = ['Группа компаний Азот', 'https://kao-azot.com', 'NaN', 'NaN', 'NaN', 'NaN']

In [290]:
#31
text_esg_krata = parse('https://krata.ru/evolution/')
text_e_krata = parse('https://krata.ru/evolution/ecology/')

text_s1_krata = parse('https://krata.ru/evolution/social-infrastructure/')
text_s2_krata = parse('https://krata.ru/cooperation/ust-zakupki/')
text_s_krata = text_s1_krata + text_s2_krata

text_g1_krata = parse('https://krata.ru/about/leadership/')
text_g2_krata = parse('https://krata.ru/about/mission/')
text_g3_krata = parse('https://krata.ru/evolution/politics/')
text_g4_krata = parse('https://krata.ru/about/ethics/')
text_g_krata = text_g1_krata + text_g2_krata + text_g3_krata + text_g4_krata

data.loc[len(data.index)] = ['Пигмент (КРАТА)', 'https://krata.ru', text_esg_krata, text_e_krata, text_s_krata, text_g_krata, 'AAA']

In [291]:
#32
data_error.loc[len(data_error.index)] = ['"Каустик" (г. Волгоград)', 'https://www.kaustik.ru/ru/', 'NaN', 'NaN', 'NaN', 'NaN']
data_pdf.loc[len(data_pdf.index)] = ['"Каустик" (г. Волгоград)', 'https://www.kaustik.ru/ru/', 'sustainability-report-toaz.pdf', 'NaN', 'NaN', 'NaN']

In [292]:
#33
text_esg_metafrax = parse('https://metafrax.ru/responsibility')
text_e_metafrax= parse('https://metafrax.ru/responsibility/protecting-the-environment')

text_s1_metafrax = parse('https://metafrax.ru/responsibility/cultural-activities')
text_s2_metafrax = parse('https://metafrax.ru/responsibility/sponsorship-charity')
text_s3_metafrax = parse('https://metafrax.ru/tenders')
text_s_metafrax = text_s1_metafrax + text_s2_metafrax + text_s3_metafrax

text_g1_metafrax = parse('https://metafrax.ru/management#text')
text_g2_metafrax = parse('https://metafrax.ru/mission')
text_g3_metafrax = parse('https://metafrax.ru/responsibility/anti-corruption-programs')
text_g_metafrax = text_g1_metafrax + text_g2_metafrax + text_g3_metafrax

data.loc[len(data.index)] = ['«Метафракс кемикалс»', 'https://metafrax.ru', text_esg_metafrax, text_e_metafrax, text_s_metafrax, text_g_metafrax, 'AAA']

In [293]:
#34
text_esg_soda = parse('https://www.soda.ru/ru/pages/detail/social_responsibility/')
text_e_soda = parse('https://www.soda.ru/ru/pages/detail/Ecology/')

text_s1_soda = parse('https://www.soda.ru/ru/pages/detail/social_responsibility/')
text_s2_soda = parse('https://www.soda.ru/ru/pages/detail/Procurement/')
text_s_soda = text_s1_soda + text_s2_soda

data.loc[len(data.index)] = ['«Башкирская содовая компания»', 'https://www.soda.ru/ru/', text_esg_soda, text_e_soda, text_s_soda, 'NaN', 'AAA']

In [294]:
#35
text_esg_kuazot = parse('https://www.kuazot.ru/news/426-kuybyshevazot-voshel-v-top-3-reytinga-socialnoy-ef/')
text_e_kuazot = parse('https://www.kuazot.ru/responsibility/ecology_and_resource_saving/')

text_s1_kuazot = parse('https://www.kuazot.ru/products/products_in_life/')
text_s2_kuazot = parse('https://www.kuazot.ru/responsibility/external_social_investment/')
text_s3_kuazot = parse('https://www.kuazot.ru/purchases/index/')
text_s_kuazot = text_s1_kuazot + text_s2_kuazot + text_s3_kuazot

text_g1_kuazot = parse('https://www.kuazot.ru/company/control/')
text_g2_kuazot = parse('https://www.kuazot.ru/company/mission_and_goals/')
text_g3_kuazot = parse('https://www.kuazot.ru/responsibility/anti_corruption/')
text_g_kuazot = text_g1_kuazot + text_g2_kuazot + text_g3_kuazot

data.loc[len(data.index)] = ['«КуйбышевАзот»', 'https://www.kuazot.ru', text_esg_kuazot, text_e_kuazot, text_s_kuazot, text_g_kuazot, 'AAA']

In [295]:
#36
text_esg_omsk = parse('https://omskcarbongroup.com/sustainability/')
text_e_omsk = parse('https://omskcarbongroup.com/sustainability/health/')

text_s1_omsk = parse('https://omskcarbongroup.com/sustainability/health/')
text_s2_omsk = parse('https://omskcarbongroup.com/tenders/')
text_s_omsk = text_s1_omsk + text_s2_omsk

data.loc[len(data.index)] = ['Омск Карбон Групп', 'https://omskcarbongroup.com/#0', text_esg_omsk, text_e_omsk, text_s_omsk, 'NaN', 'AAA']

In [296]:
#37
data_error.loc[len(data_error.index)] = ['«КамАЗ»', 'https://kamaz.ru', 'NaN', 'NaN', 'NaN', 'NaN']
data_pdf.loc[len(data_pdf.index)] = ['«КамАЗ»', 'https://kamaz.ru', 'Strategy KAMAZ 2030_v 2024_ru_public.pdf', 'NaN', 'NaN', 'NaN']

In [297]:
#38
text_esg_tmholding = parse('https://tmholding.ru/improvement/')
text_e_tmholding = parse('https://tmholding.ru/about_us/okhrana-truda/')

text_s_tmholding = parse('https://tmholding.ru/engineering/#life')

text_g_tmholding = parse('https://tmholding.ru/about_us/ethics/')

data.loc[len(data.index)] = ['«Трансмашхолдинг»', 'https://tmholding.ru/', text_esg_tmholding, text_e_tmholding, text_s_tmholding, text_g_tmholding, 'AAA']

In [298]:
#39
text_s1_aoosk = parse('https://aoosk.ru/personnel-policy/employees/')
text_s2_aoosk = parse('https://aoosk.ru/purchases/')
text_s_aoosk = text_s1_aoosk + text_s2_aoosk

text_g1_aoosk = parse('https://aoosk.ru/about/board-directors/')
text_g2_aoosk = parse('https://aoosk.ru/about/anti-corruption/')
text_g_aoosk = text_g1_aoosk + text_g2_aoosk

data.loc[len(data.index)] = ['«Объединенная судостроительная корпорация»', 'https://aoosk.ru', 'NaN', 'NaN', text_s_aoosk, text_g_aoosk, 'AAA']

In [299]:
#40
data_error.loc[len(data_error.index)] = ['ГАЗ', 'http://www.gaz.ru', 'NaN', 'NaN', 'NaN', 'NaN']

In [300]:
#41
text_esg_sinara = parse('https://www.sinara-group.com/responsibility/charity/')
text_s_sinara = parse('https://www.sinara-group.com/career/life/')
text_g_sinara = parse('https://www.sinara-group.com/responsibility/komplaens/')

data.loc[len(data.index)] = ['«Группа Синара»', 'https://www.sinara-group.com', text_esg_sinara, 'NaN', text_s_sinara, text_g_sinara, 'AAA']

In [301]:
#42
data_error.loc[len(data_error.index)] = ['Кордиант', 'https://cordiant.ru', 'NaN', 'NaN', 'NaN', 'NaN']

In [302]:
#43
data_error.loc[len(data_error.index)] = ['«Объединенная авиастроительная корпорация»', 'https://www.uacrussia.ru/', 'NaN', 'NaN', 'NaN', 'NaN']

In [303]:
#44
text_e_nordsy = parse('https://nordsy.spb.ru/about/qms/ecology/index.php')

text_s1_nordsy = parse('https://nordsy.spb.ru/job/guarantees/')
text_s2_nordsy = parse('https://nordsy.spb.ru/bidding-and-procurement/trades/')
text_s_nordsy = text_s1_nordsy + text_s2_nordsy

text_g1_nordsy = parse('https://nordsy.spb.ru/about/control/')
text_g2_nordsy = parse('https://nordsy.spb.ru/about/reaction/')
text_g_nordsy = text_g1_nordsy + text_g2_nordsy

data.loc[len(data.index)] = ['Северная верфь', 'https://nordsy.spb.ru/', 'NaN', text_e_nordsy, text_s_nordsy, text_g_nordsy, 'AAA']

In [304]:
#45
text_e_power = parse('https://power-m.ru/company/social-responsibility/environmental-protection/')

text_s1_power = parse('https://power-m.ru/customers/customer-account/')
text_s2_power = parse('https://power-m.ru/company/social-responsibility/')
text_s3_power = parse('https://power-m.ru/suppliers/')
text_s_power = text_s1_power + text_s2_power + text_s3_power

text_g_power = parse('https://power-m.ru/company/values/')

data.loc[len(data.index)] = ['Силовые машины', 'https://power-m.ru/', 'NaN', text_e_power, text_s_power, text_g_power, 'AAA']
data_pdf.loc[len(data_pdf.index)] = ['Силовые машины', 'https://power-m.ru/', 'NaN', 'NaN', 'NaN', 'https://power-m.ru/company/compliance-with-codes-and-regulations/Standarti%20povedenia.pdf']
data_pdf.loc[len(data_pdf.index)] = ['Силовые машины', 'https://power-m.ru/', 'NaN', 'NaN', 'NaN', 'https://power-m.ru/company/anti-corruption-policies/ppk_.pdf']

In [306]:
#46
text_s1_onegoshipyard = parse('https://onegoshipyard.ru/karera/sotsialnaya-politika/')
text_s2_onegoshipyard = parse('https://onegoshipyard.ru/zakupki/polozhenie-o-zakupkah/')
text_s_onegoshipyard = text_s1_onegoshipyard + text_s2_onegoshipyard

text_g1_onegoshipyard = parse('https://onegoshipyard.ru/about/rukovodstvo/')
text_g2_onegoshipyard = parse('https://onegoshipyard.ru/about/')
text_g3_onegoshipyard = parse('https://onegoshipyard.ru/about/protivodeystvie-korruptsii/')
text_g_onegoshipyard = text_g1_onegoshipyard + text_g2_onegoshipyard + text_g3_onegoshipyard

data.loc[len(data.index)] = ['Онежский судостроительный завод', 'https://onegoshipyard.ru/', 'NaN', 'NaN', text_s_onegoshipyard, text_g_onegoshipyard, 'AAA']

In [308]:
#47
text_s1_kirovets = parse('https://kirovets-ptz.com/company/kadrovaya-politika/')
text_s2_kirovets = parse('https://kirovets-ptz.com/company/zakupki/')
text_s_kirovets = text_s1_kirovets + text_s2_kirovets

data.loc[len(data.index)] = ['Петербургский тракторный завод', 'https://kirovets-ptz.com/', 'NaN', 'NaN', text_s_kirovets, 'NaN', 'AAA']
data_pdf.loc[len(data_pdf.index)] = ['Петербургский тракторный завод', 'https://kirovets-ptz.com/', 'NaN', 'NaN', 'NaN', 'https://kirovets-ptz.com/upload/medialibrary/734/cn2omv7pehmidd7gs33vgjz508fi8upu/%D0%9F%D0%BE%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%D0%B0%20%D0%B2%20%D0%BE%D0%B1%D0%BB%D0%B0%D1%81%D1%82%D0%B8%20%D0%BF%D1%80%D0%BE%D1%82%D0%B8%D0%B2%D0%BE%D0%B4%D0%B5%D0%B9%D1%81%D1%82%D0%B2%D0%B8%D1%8F%20%D0%BA%D0%BE%D1%80%D0%BF%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D0%BC%D1%83%20%D0%BC%D0%BE%D1%88%D0%B5%D0%BD%D0%BD%D0%B8%D1%87%D0%B5%D1%81%D1%82%D0%B2%D1%83.pdf']

In [310]:
#48
data_error.loc[len(data_error.index)] = ['ЕВРАЗ', 'https://www.evraz.com/ru/', 'NaN', 'NaN', 'NaN', 'NaN']

In [314]:
#49
data_error.loc[len(data_error.index)] = ['«Северсталь»', 'https://severstal.com/rus/', 'NaN', 'NaN', 'NaN', 'NaN']

In [317]:
#50
text_esg_mmk = parse('https://mmk.ru/ru/sustainability/')
text_e_mmk = parse('https://mmk.ru/ru/sustainability/ecology/')

text_s1_mmk = parse('https://mmk.ru/ru/career/')
text_s2_mmk = parse('https://mmk.ru/ru/for-suppliers/')
text_s_mmk = text_s1_mmk + text_s2_mmk

text_g1_mmk = parse('https://mmk.ru/ru/about/corporate-governance/')
text_g2_mmk = parse('https://mmk.ru/ru/about/strategy/')
text_g3_mmk = parse('https://mmk.ru/ru/about/corporate-governance/internal-documents/')
text_g_mmk = text_g1_mmk + text_g2_mmk + text_g3_mmk

data.loc[len(data.index)] = ['Магнитогорский металлургический комбинат (ММК)', 'https://mmk.ru/ru/', text_esg_mmk, text_e_mmk, text_s_mmk, text_g_mmk, 'AAA']

In [327]:
#51
text_esg_nkmk = parse('https://nlmk.com/ru/sustainability/')
text_e_nkmk = parse('https://nlmk.com/ru/sustainability/environment/')

text_s1_nkmk = parse('https://career.nlmk.com/')
text_s2_nkmk = parse('https://nlmk.com/ru/for-suppliers-and-contractors/')
text_s_nkmk = text_s1_nkmk + text_s2_nkmk

text_g1_nkmk = parse('https://nlmk.com/ru/about/governance/')
text_g2_nkmk = parse('https://nlmk.com/ru/sustainability/anticorruption/anti-corruption-compliance/')
text_g_nkmk = text_g1_nkmk + text_g2_nkmk 

data.loc[len(data.index)] = ['НЛМК', 'https://nlmk.com/ru/', 'NaN', 'NaN', text_s_nkmk, 'NaN', 'AAA']

In [328]:
#52
data_error.loc[len(data_error.index)] = ['«Металлоинвест»', 'https://www.metalloinvest.com', 'NaN', 'NaN', 'NaN', 'NaN']

In [331]:
#53
text_esg_tmk = parse('https://www.tmk-group.ru/sustainable-development')
text_e_tmk = parse('https://www.tmk-group.ru/esg-ecology')

text_s1_tmk = parse('https://www.tmk-group.ru/careers')
text_s2_tmk = parse('https://zakupki.tmk-group.com/#tmk/front/index')
text_s_tmk = text_s1_tmk + text_s2_tmk

text_g1_tmk = parse('https://www.tmk-group.ru/GM_Summary')
text_g2_tmk = parse('https://www.tmk-group.ru/Our_mission')
text_g3_tmk = parse('https://www.tmk-group.ru/esg-docs')
text_g_tmk = text_g1_tmk + text_g2_tmk + text_g3_tmk

data.loc[len(data.index)] = ['ТМК (Трубная металлургическая компания)', 'https://www.tmk-group.ru', text_esg_tmk, text_e_tmk, text_s_tmk, text_g_tmk, 'AAA']

In [335]:
#54
text_esg_seligdar = parse('https://seligdar.ru/sustainability/principles-approaches/')
text_e_seligdar = parse('https://seligdar.ru/sustainability/environment/')

text_s1_seligdar = parse('https://seligdar.ru/career/working-with-us/')
text_s2_seligdar = parse('https://seligdar.ru/suppliers/procurement-principles/')
text_s_seligdar = text_s1_seligdar + text_s2_seligdar 

text_g1_seligdar = parse('https://seligdar.ru/company/mission-and-strategy/')
text_g2_seligdar = parse('https://seligdar.ru/investors/disclosure/official-documents/')
text_g_seligdar = text_g1_seligdar + text_g2_seligdar

data.loc[len(data.index)] = ['ПАО «Селигдар»', 'https://seligdar.ru', text_esg_seligdar, text_e_seligdar, text_s_seligdar, text_g_seligdar, 'AAA']

In [339]:
#55
data_error.loc[len(data_error.index)] = ['«Мечел»', 'https://www.mechel.ru/#1', 'NaN', 'NaN', 'NaN', 'NaN']

In [336]:
data

,Название,Ссылка,"Устойчивое развитие, Социальная отчетность, ESG",E,S,G,оценка
0,Адмиралтейские верфи,http://admship.ru/,NaN,NaN,Карьера\nКОНТАКТЫ\nEng\nКомпания\nИстория комп...,Противодействие коррупции\nКОНТАКТЫ\nEng\nКомп...,AAA
1,Алмаз-Антей,http://www.almaz-antey.ru/,NaN,NaN,Акционерное общество «Концерн ВКО «Алмаз - Ант...,Акционерное общество «Концерн ВКО «Алмаз - Ант...,AAA
2,Армалит,https://armalit.ru/,NaN,NaN,Учебный центр\nСкачать каталог\nСудостроение\n...,Руководство завода\nСкачать каталог\nСудострое...,AAA
3,Аттика,https://attikarus.ru/,NaN,NaN,Поставщикам\nRu\nEng\n8 800 234 03 83\nнаша пр...,Команда\nRu\nEng\n8 800 234 03 83\nнаша продук...,AAA
4,Балтийский завод,https://www.bz.ru/,NaN,NaN,Социальная политика\nБалтийский завод\nО завод...,Руководство\nБалтийский завод\nО заводе\nИстор...,AAA
5,Бронка Групп,https://www.bronkagroup.ru/,NaN,Социальная ответственность\nО компании\nО нас\...,Работа в компании | Бронка Групп\nО компании\n...,Руководство холдинга | Бронка Групп\nО компани...,AAA
6,Всеволожский крановый завод,https://werker.ru/,NaN,NaN,Вакансии\nОборудование\nДвухбалочные мостовые ...,Подъемно-транспортное оборудование Всеволожско...,AAA
7,Гидроприбор,https://www.gidropribor.ru/,NaN,NaN,﻿\nКорпоративная культура\nО корпорации\nКонта...,﻿\nПартнерам - Антикоррупционная политика - АО...,AAA
8,Гранит-Электрон,https://www.granit-electron.ru/,NaN,NaN,Оплата труда и социальный пакет\nО корпорации\...,Документы\nО корпорации\nКонтактная информация...,AAA
9,Императорский Фарфоровый завод,https://www.ipm.ru/,NaN,NaN,NaN,NaN,AAA
